In [1]:
import ROOT

# Open the ROOT file (use the correct file extension, e.g., .root if it's a ROOT file)
file = ROOT.TFile("DataMuons.root")

# List the contents of the file
file.ls()

# Assuming there is only one tree, or you know the tree name after listing the contents
tree_name = None

# Loop through the keys in the file to find a TTree object
for key in file.GetListOfKeys():
    obj = key.ReadObj()
    if isinstance(obj, ROOT.TTree):
        tree_name = obj.GetName()
        print(f"Found tree: {tree_name}")
        break

if tree_name:
    tree = file.Get(tree_name)

    # Print the branches of the tree to identify available branches
    print("Branches in the tree:")
    tree.Print()

    # Example: Print first 10 entries of a specific branch
    for i in range(10):
        tree.GetEntry(i)
        # Replace 'muon_pt' with the actual branch name after inspecting the tree
        # For now, let's just print out all the branches in the first entry
        print(f"Entry {i}:")
        for branch in tree.GetListOfBranches():
            branch_name = branch.GetName()
            value = getattr(tree, branch_name)
            print(f"  {branch_name}: {value}")
else:
    print("No TTree found in the ROOT file.")


Welcome to JupyROOT 6.28/00
TFile**		DataMuons.root	
 TFile*		DataMuons.root	
  KEY: TTree	mini;1	4-vectors + variables required for scaling factors
Found tree: mini
Branches in the tree:
******************************************************************************
*Tree    :mini      : 4-vectors + variables required for scaling factors     *
*Entries :  7028084 : Total =      1739213979 bytes  File  Size =  619767110 *
*        :          : Tree compression factor =   2.81                       *
******************************************************************************
*Br    0 :runNumber : runNumber/I                                            *
*Entries :  7028084 : Total  Size=   28115633 bytes  File Size  =     139881 *
*Baskets :       31 : Basket Size=    1312768 bytes  Compression= 200.99     *
*............................................................................*
*Br    1 :eventNumber : eventNumber/I                                        *
*Entries :  7028084 : 

In [2]:
import ROOT


file = ROOT.TFile("DataMuons.root")
tree = file.Get("mini")  #  'tree_name'=mini

# Print the contents of the file
file.ls()

# Example: Print first 10 entries of the tree
for i in range(10):
    tree.GetEntry(i)
    print(tree.lep_pt)  


TFile**		DataMuons.root	
 TFile*		DataMuons.root	
  OBJ: TTree	mini	4-vectors + variables required for scaling factors : 0 at: 0x5dcb609ac020
  KEY: TTree	mini;1	4-vectors + variables required for scaling factors


In [3]:
import ROOT

file = ROOT.TFile("DataMuons.root")
tree = file.Get("mini")  # 'tree_name'=mini

# List the contents of the file
file.ls()

# Print first 10 entries of the tree
for i in range(10):
    tree.GetEntry(i)
    print(f"Entry {i}:")
    for pt in tree.lep_pt:
        print(f"  lep_pt: {pt}")


Entry 0:
  lep_pt: 40531.85546875
Entry 1:
  lep_pt: 37172.48828125
Entry 2:
  lep_pt: 41404.36328125
Entry 3:
  lep_pt: 36330.359375
Entry 4:
  lep_pt: 29865.91796875
Entry 5:
  lep_pt: 39808.7109375
Entry 6:
  lep_pt: 25943.61328125
Entry 7:
  lep_pt: 38542.88671875
Entry 8:
  lep_pt: 32593.083984375
Entry 9:
  lep_pt: 32656.75
TFile**		DataMuons.root	
 TFile*		DataMuons.root	
  OBJ: TTree	mini	4-vectors + variables required for scaling factors : 0 at: 0x5dcb609ac020
  KEY: TTree	mini;1	4-vectors + variables required for scaling factors


In [4]:
import ROOT

# Function to read the tree and fill histograms
def readTreeToMllHistogram(tree, dataType):
    if dataType not in ["data", "MC"]:
        print(f"The data type should be \"data\" or \"MC\", not {dataType}")
        return None

    mll = ROOT.TH1D(dataType, f"m_{{ll}}, {dataType}", 150, 50.e3, 200.e3)
    mll.Sumw2()

    for entryNum in range(tree.GetEntries()):
        tree.GetEntry(entryNum)
        if getattr(tree, "lep_n") != 2:
            continue

        pT = getattr(tree, "lep_pt")
        eta = getattr(tree, "lep_eta")
        phi = getattr(tree, "lep_phi")
        nrg = getattr(tree, "lep_E")

        lepton0 = ROOT.TLorentzVector()
        lepton0.SetPtEtaPhiE(pT[0], eta[0], phi[0], nrg[0])
        lepton1 = ROOT.TLorentzVector()
        lepton1.SetPtEtaPhiE(pT[1], eta[1], phi[1], nrg[1])

        dilepton = lepton0 + lepton1

        weight = 1
        if dataType == "MC":
            weight *= getattr(tree, "mcWeight")
            weight *= getattr(tree, "scaleFactor_PILEUP")
            weight *= getattr(tree, "scaleFactor_MUON")
            weight *= getattr(tree, "scaleFactor_TRIGGER")

        mll.Fill(dilepton.M(), weight)

    return mll

# Specify input and output file names
dataFileName = "DataMuons.root"
mcFileName = "mc_147771.Zmumu.root"
histFileName = "output_large_data.root"

# Process data file
print("Running over data...")
dataFile = ROOT.TFile(dataFileName, "READ")
dataTree = dataFile.Get("mini")
dataHisto = readTreeToMllHistogram(dataTree, "data")
dataHisto.SetDirectory(0)
dataFile.Close()
print("Done running over data")

# Process MC file
print("Running over MC...")
mcFile = ROOT.TFile(mcFileName, "READ")
mcTree = mcFile.Get("mini")
mcHisto = readTreeToMllHistogram(mcTree, "MC")
mcHisto.SetDirectory(0)
mcFile.Close()
print("Done running over MC")

# Write histograms to output file
print("Writing histograms to output file...")
outHistFile = ROOT.TFile.Open(histFileName, "RECREATE")
outHistFile.cd()
dataHisto.Write()
mcHisto.Write()
outHistFile.Close()
print("Done writing histograms to output file")

#removing auto statstics info at right cornor
mcHisto.SetStats(0)
dataHisto.SetStats(0)

# Plotting the histograms
canvas = ROOT.TCanvas("canvas", "canvas", 800, 600)
canvas.SetLogy(True)

# Normalize MC histogram to data
mcHisto.Scale(dataHisto.Integral() / mcHisto.Integral())

# Set styles for histograms
dataHisto.SetLineColor(ROOT.kBlack)
mcHisto.SetLineColor(ROOT.kRed)
dataHisto.SetLineWidth(2)
mcHisto.SetLineWidth(2)

# Draw MC histogram
mcHisto.Draw("hist")
# Draw data histogram on the same canvas
dataHisto.Draw("same pe")

# Add legend
legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)
legend.AddEntry(dataHisto, "Data", "pe")
legend.AddEntry(mcHisto, "MC", "l")
legend.Draw()

# Save the canvas as an image file
canvas.SaveAs("muon_muon_comparison_plot.png")

print("Plot saved as comparison_plot.png")


Running over data...
Done running over data
Running over MC...
Done running over MC
Writing histograms to output file...
Done writing histograms to output file
Plot saved as comparison_plot.png


Info in <TCanvas::Print>: png file muon_muon_comparison_plot.png has been created


In [5]:
import ROOT

# Function to read the tree and fill histograms
def readTreeToMllHistogram(tree, dataType):
    if dataType not in ["data", "MC"]:
        print(f"The data type should be \"data\" or \"MC\", not {dataType}")
        return None
    
    mll = ROOT.TH1D(dataType, f"m_{{ll}}, {dataType}", 150, 50.e3, 200.e3)
    mll.Sumw2()

    for entryNum in range(tree.GetEntries()):
        tree.GetEntry(entryNum)
        if getattr(tree, "lep_n") != 2:
            continue

        pT = getattr(tree, "lep_pt")
        eta = getattr(tree, "lep_eta")
        phi = getattr(tree, "lep_phi")
        nrg = getattr(tree, "lep_E")

        lepton0 = ROOT.TLorentzVector()
        lepton0.SetPtEtaPhiE(pT[0], eta[0], phi[0], nrg[0])
        lepton1 = ROOT.TLorentzVector()
        lepton1.SetPtEtaPhiE(pT[1], eta[1], phi[1], nrg[1])

        dilepton = lepton0 + lepton1

        weight = 1
        if dataType == "MC":
            weight *= getattr(tree, "mcWeight")
            weight *= getattr(tree, "scaleFactor_PILEUP")
            weight *= getattr(tree, "scaleFactor_MUON")
            weight *= getattr(tree, "scaleFactor_TRIGGER")

        mll.Fill(dilepton.M(), weight)

    return mll

# Specify input and output file names
dataFileName = "DataMuons.root"
mcFileName = "mc_147771.Zmumu.root"
histFileName = "output_large_data.root"

# Process data file
print("Running over data...")
dataFile = ROOT.TFile(dataFileName, "READ")
dataTree = dataFile.Get("mini")
dataHisto = readTreeToMllHistogram(dataTree, "data")
dataHisto.SetDirectory(0)
dataFile.Close()
print("Done running over data")

# Process MC file
print("Running over MC...")
mcFile = ROOT.TFile(mcFileName, "READ")
mcTree = mcFile.Get("mini")
mcHisto = readTreeToMllHistogram(mcTree, "MC")
mcHisto.SetDirectory(0)
mcFile.Close()
print("Done running over MC")

# Write histograms to output file
print("Writing histograms to output file...")
outHistFile = ROOT.TFile.Open(histFileName, "RECREATE")
outHistFile.cd()
dataHisto.Write()
mcHisto.Write()
outHistFile.Close()
print("Done writing histograms to output file")

# Plotting the histograms
canvas = ROOT.TCanvas("canvas", "canvas", 800, 600)
canvas.SetLogy(True)

dataHisto.SetStats(0)
mcHisto.SetStats(0)
dataHisto.SetLineColor(ROOT.kBlack)
mcHisto.SetLineColor(ROOT.kRed)
dataHisto.SetLineWidth(2)
mcHisto.SetLineWidth(2)
mcHisto.GetYaxis().SetTitle("Number of events")
dataHisto.GetYaxis().SetTitle("Number of events")
mcHisto.GetXaxis().SetTitle("m_{ll} [MeV]")
dataHisto.GetXaxis().SetTitle("m_{ll} [MeV]")

# Open the canvas for continuous printing
canvas.Print("muon_muon_comparison_plots.pdf[")

# Draw data histogram
dataHisto.Draw("pe")
canvas.Print("muon_muon_comparison_plots.pdf")

# Draw MC histogram (un-normalized)
mcHisto.Draw("hist")
canvas.Print("muon_muon_comparison_plots.pdf")

# Normalize MC histogram
mcHistoNorm = mcHisto.Clone("mcHistoNorm")
mcHistoNorm.Scale(dataHisto.Integral() / mcHisto.Integral())

# Draw the combined histogram with normalized MC
mcHistoNorm.Draw("hist")
dataHisto.Draw("same, pe")
canvas.Print("muon_muon_comparison_plots.pdf")

# Plot their ratio
ratio = dataHisto.Clone()
ratio.Divide(mcHistoNorm)
ratio.SetLineColor(ROOT.kRed)

# Draw the normal plots (not the ratio)
pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 1)
pad1.SetLogy(True)
pad1.SetBottomMargin(0)
pad1.Draw()
pad1.cd()
mcHistoNorm.SetTitle("")
mcHistoNorm.GetXaxis().SetLabelSize(0)
mcHistoNorm.GetXaxis().SetTitleSize(0)
mcHistoNorm.GetYaxis().SetTitleSize(0.05)
mcHistoNorm.Draw("hist")
dataHisto.Draw("pe,same")

legend = ROOT.TLegend(0.7, 0.6, 0.85, 0.75)
legend.AddEntry(mcHistoNorm, "MC (normalized)")
legend.AddEntry(dataHisto, "Data")
legend.SetLineWidth(0)
legend.Draw("same")

latex = ROOT.TLatex()
latex.SetNDC()
latex.SetTextSize(0.06)
latex.DrawText(0.7, 0.83, "Z#mu#mu + Jets,  2016 ATLAS open data ")
latex.SetTextSize(0.04)
latex.DrawText(0.7, 0.77, "Di-muon events")

canvas.Print("muon_muon_comparison_plots.pdf")

# Now draw the ratio
canvas.cd()
pad2 = ROOT.TPad("pad2", "pad2", 0, 0.05, 1, 0.3)
pad2.SetTopMargin(0)
pad2.SetBottomMargin(0.25)
pad2.Draw()
pad2.cd()
ratio.SetTitle("")
ratio.GetXaxis().SetLabelSize(0.12)
ratio.GetXaxis().SetTitleSize(0.12)
ratio.GetYaxis().SetLabelSize(0.1)
ratio.GetYaxis().SetTitleSize(0.15)
ratio.GetYaxis().SetTitle("Data/MC")
ratio.GetYaxis().SetTitleOffset(0.3)
ratio.GetYaxis().SetRangeUser(0.5, 1.5)
ratio.GetYaxis().SetNdivisions(207)
ratio.Draw("pe")

line = ROOT.TLine(50.e3, 1, 200.e3, 1)
line.SetLineColor(ROOT.kBlack)
line.SetLineWidth(2)
line.Draw("same")

canvas.Print("muon_muon_comparison_plots.pdf")

# Close the output PDF
canvas.Print("muon_muon_comparison_plots.pdf]")

print("Plots saved as comparison_plots.pdf")


Running over data...
Done running over data
Running over MC...
Done running over MC
Writing histograms to output file...
Done writing histograms to output file
Plots saved as comparison_plots.pdf


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file muon_muon_comparison_plots.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file muon_muon_comparison_plots.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file muon_muon_comparison_plots.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file muon_muon_comparison_plots.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file muon_muon_comparison_plots.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file muon_muon_comparison_plots.pdf
Info in <TCanvas::Print>: pdf file muon_muon_comparison_plots.pdf has been closed


In [6]:
import ROOT

# Function to read the tree and fill histograms
def readTreeToMllHistogram(tree, dataType):
    if dataType not in ["data", "MC"]:
        print(f"The data type should be \"data\" or \"MC\", not {dataType}")
        return None
    
    mll = ROOT.TH1D(dataType, f"m_{{ll}}, {dataType}", 150, 50.e3, 200.e3)
    mll.Sumw2()

    for entryNum in range(tree.GetEntries()):
        tree.GetEntry(entryNum)
        if getattr(tree, "lep_n") != 2:
            continue

        pT = getattr(tree, "lep_pt")
        eta = getattr(tree, "lep_eta")
        phi = getattr(tree, "lep_phi")
        nrg = getattr(tree, "lep_E")

        lepton0 = ROOT.TLorentzVector()
        lepton0.SetPtEtaPhiE(pT[0], eta[0], phi[0], nrg[0])
        lepton1 = ROOT.TLorentzVector()
        lepton1.SetPtEtaPhiE(pT[1], eta[1], phi[1], nrg[1])

        dilepton = lepton0 + lepton1

        weight = 1
        if dataType == "MC":
            weight *= getattr(tree, "mcWeight")
            weight *= getattr(tree, "scaleFactor_PILEUP")
            weight *= getattr(tree, "scaleFactor_MUON")
            weight *= getattr(tree, "scaleFactor_TRIGGER")

        mll.Fill(dilepton.M(), weight)

    return mll

# Specify input and output file names
dataFileName = "Data.root"
mcFileName = "MC.root"
histFileName = "output_histograms.root"

# Process data file
print("Running over data...")
dataFile = ROOT.TFile(dataFileName, "READ")
dataTree = dataFile.Get("HASCO")
dataHisto = readTreeToMllHistogram(dataTree, "data")
dataHisto.SetDirectory(0)
dataFile.Close()
print("Done running over data")

# Process MC file
print("Running over MC...")
mcFile = ROOT.TFile(mcFileName, "READ")
mcTree = mcFile.Get("HASCO")
mcHisto = readTreeToMllHistogram(mcTree, "MC")
mcHisto.SetDirectory(0)
mcFile.Close()
print("Done running over MC")

# Write histograms to output file
print("Writing histograms to output file...")
outHistFile = ROOT.TFile.Open(histFileName, "RECREATE")
outHistFile.cd()
dataHisto.Write()
mcHisto.Write()
outHistFile.Close()
print("Done writing histograms to output file")

# Plotting the histograms
canvas = ROOT.TCanvas("canvas", "canvas", 800, 600)
canvas.SetLogy(True)

dataHisto.SetStats(0)
mcHisto.SetStats(0)
dataHisto.SetLineColor(ROOT.kBlack)
mcHisto.SetLineColor(ROOT.kRed)
dataHisto.SetLineWidth(2)
mcHisto.SetLineWidth(2)
mcHisto.GetYaxis().SetTitle("Number of events")
dataHisto.GetYaxis().SetTitle("Number of events")
mcHisto.GetXaxis().SetTitle("m_{ll} [MeV]")
dataHisto.GetXaxis().SetTitle("m_{ll} [MeV]")

# Open the canvas for continuous printing
canvas.Print("comparison_plots .pdf[")

# Draw data histogram
dataHisto.Draw("pe")
canvas.Print("comparison_plots.pdf")

# Draw MC histogram (un-normalized)
mcHisto.Draw("hist")
canvas.Print("comparison_plots.pdf")

# Normalize MC histogram
mcHistoNorm = mcHisto.Clone("mcHistoNorm")
mcHistoNorm.Scale(dataHisto.Integral() / mcHisto.Integral())

# Draw the combined histogram with normalized MC
mcHistoNorm.Draw("hist")
dataHisto.Draw("same, pe")
canvas.Print("comparison_plots.pdf")

# Plot their ratio
ratio = dataHisto.Clone()
ratio.Divide(mcHistoNorm)
ratio.SetLineColor(ROOT.kRed)

# Draw the normal plots (not the ratio)
pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 1) # Set the y-axis of the top plot to be 
pad1.SetLogy(True)
pad1.SetBottomMargin(0) # Upper and lower pads are joined
pad1.Draw()             # Draw the upper pad in the canvas
pad1.cd()               # pad1 becomes the current pad
mcHistoNorm.SetTitle("")# pad1 becomes the current pad
mcHistoNorm.GetXaxis().SetLabelSize(0)  # Remove x-axis labels for the top pad
mcHistoNorm.GetXaxis().SetTitleSize(0)  # Remove x-axis title for the top pad
mcHistoNorm.GetYaxis().SetTitleSize(0.05) #Increase y-axis title size (pad is not full page)
mcHistoNorm.Draw("hist")
dataHisto.Draw("pe,same") # Draw the data points on top of the MC histo

legend = ROOT.TLegend(0.7, 0.6, 0.85, 0.75)  
legend.AddEntry(mcHistoNorm, "MC (normalized)")
legend.AddEntry(dataHisto, "Data")
legend.SetLineWidth(0) 
legend.Draw("same")

latex = ROOT.TLatex()
latex.SetNDC()
latex.SetTextSize(0.06)
latex.DrawText(0.7, 0.83, "HASCO 2018")
latex.SetTextSize(0.04)
latex.DrawText(0.7, 0.77, "Di-muon events")

# Now draw the ratio
canvas.cd()  # Go back to the main canvas before defining pad2
pad2 = ROOT.TPad("pad2", "pad2", 0, 0.05, 1, 0.3)
pad2.SetTopMargin(0)       # Upper and lower pads are joined
pad2.SetBottomMargin(0.25) # Expand the bottom margin for extra label space
pad2.Draw()                # Draw the lower pad in the canvas
pad2.cd()                  # pad2 becomes the current pad
ratio.SetTitle("")         # Turn off the title to avoid overlap
ratio.GetXaxis().SetLabelSize(0.12) # Larger x-axis labels (pad is not full page)
ratio.GetXaxis().SetTitleSize(0.12)
ratio.GetYaxis().SetLabelSize(0.1)
ratio.GetYaxis().SetTitleSize(0.15)
ratio.GetYaxis().SetTitle("Data/MC") # Change the y-axis title (this is the ratio)
ratio.GetYaxis().SetTitleOffset(0.3) # Reduce the y-axis title spacing
ratio.GetYaxis().SetRangeUser(0.5, 1.5) # Set the y-axis ratio range from 0.5 to 1.5
ratio.GetYaxis().SetNdivisions(207) # Change the y-axis tick-marks to work better
ratio.Draw("pe")    # Draw the ratio in the current pad

line = ROOT.TLine(50.e3, 1, 200.e3, 1)
line.SetLineColor(ROOT.kBlack)
line.SetLineWidth(2)
line.Draw("same")

canvas.Print("comparison_plots.pdf")

# Fit a Gaussian to the histogram
gaussFit = ROOT.TF1("gaussfit", "gaus", 81.e3, 101.e3)
dataHisto.Fit(gaussFit, "E")
dataHisto.Draw("pe")
gaussFit.Draw("same")
canvas.Print("comparison_plots.pdf")

# Add the fit results to the plot
latex.SetTextSize(0.03)
chi2 = gaussFit.GetChisquare()
ndof = gaussFit.GetNDF()
latex.DrawText(0.5, 0.80, "Mean = %.1f GeV" % (gaussFit.GetParameter(1) / 1000))
latex.DrawText(0.5, 0.75, "Width = %.1f GeV" % (gaussFit.GetParameter(2) / 1000))
latex.DrawText(0.5, 0.70, "chi2/ndof = %.1f/%d = %.1f" % (chi2, ndof, chi2 / ndof))
canvas.Print("comparison_plots.pdf")

# Fit a Breit-Wigner to the Z peak
k_num = "2*sqrt(2)*[0]*[1]*sqrt([0]*[0]*([0]*[0]+[1]*[1]))"
k_denom = "3.14159*sqrt([0]*[0] + sqrt([0]*[0]*([0]*[0]+[1]*[1])))"
denom = "((x*x-[0]*[0])*(x*x-[0]*[0]) + [0]*[0]*[1]*[1])"
bwFit = ROOT.TF1("bwfit", "[2]*(%s/%s)/%s" % (k_num, k_denom, denom), 81.e3, 101.e3)
bwFit.SetParameter(0, 91.1876e3) # Mass of Z boson in MeV
bwFit.SetParameter(1, 2.4952e3)  # Width of Z boson in MeV
dataHisto.Fit(bwFit, "E")
dataHisto.Draw("pe")
bwFit.Draw("same")
canvas.Print("comparison_plots.pdf")

# Add the fit results to the plot
latex.SetTextSize(0.03)
chi2 = bwFit.GetChisquare()
ndof = bwFit.GetNDF()
latex.DrawText(0.5, 0.80, "Mean = %.1f GeV" % (bwFit.GetParameter(0) / 1000))
latex.DrawText(0.5, 0.75, "Width = %.1f GeV" % (bwFit.GetParameter(1) / 1000))
latex.DrawText(0.5, 0.70, "chi2/ndof = %.1f/%d = %.1f" % (chi2, ndof, chi2 / ndof))
canvas.Print("comparison_plots.pdf")

# Close the canvas for continuous printing
canvas.Print("comparison_plots.pdf]")

print("Done with plotting and fitting")


Running over data...
Done running over data
Running over MC...
Done running over MC
Writing histograms to output file...
Done writing histograms to output file
Done with plotting and fitting
 FCN=10889 FROM MINOS     STATUS=SUCCESSFUL     20 CALLS         225 TOTAL
                     EDM=6.81397e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     6.26913e+03   4.30927e+01   5.26908e-02  -2.92835e-06
   2  Mean         9.06248e+04   1.47209e+01  -1.18182e-02  -7.20941e-07
   3  Sigma        3.29138e+03   1.73890e+01   1.73890e+01  -4.18851e-02
 FCN=1841.93 FROM MINOS     STATUS=SUCCESSFUL     20 CALLS         264 TOTAL
                     EDM=3.56586e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DE

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file comparison_plots .pdf has been created
Info in <TCanvas::Print>: pdf file comparison_plots.pdf has been created
Info in <TCanvas::Print>: pdf file comparison_plots.pdf has been created
Info in <TCanvas::Print>: pdf file comparison_plots.pdf has been created
Info in <TCanvas::Print>: pdf file comparison_plots.pdf has been created
Info in <TCanvas::Print>: pdf file comparison_plots.pdf has been created
Info in <TCanvas::Print>: pdf file comparison_plots.pdf has been created
Info in <TCanvas::Print>: pdf file comparison_plots.pdf has been created
Info in <TCanvas::Print>: pdf file comparison_plots.pdf has been created
Info in <TCanvas::Print>: pdf file comparison_plots.pdf has been created


In [7]:
plotFileName="ATLAS_2016_RUN_fitting.pdf"
# Basic style
dataHisto.SetStats(0) # Turn off the statistics box
dataHisto.SetLineColor(ROOT.kBlack) # Set the line color to black for data
dataHisto.SetLineWidth(2) # Set the line width to 2 for data
dataHisto.GetYaxis().SetTitle("Number of events") # Set y-axis label
dataHisto.GetXaxis().SetTitle("m_{ll} [MeV]") # Set x-axis label

# Prepare the canvas for plotting
canvas = ROOT.TCanvas("canvas") # Make a canvas
canvas.cd() # Move into the canvas (so anything drawn is part of this canvas)
canvas.SetLogy(True) # Set the y-axis to be logarithmic

# Open the canvas for continuous printing
canvas.Print(plotFileName + "[") # Open for multiple plots in a single pdf

# Draw the data histogram, as data Points with Errors
dataHisto.Draw("pe")
canvas.Print(plotFileName) # Write the data histogram plot to the output file

# Fit a Gaussian to the histogram
gaussFit = ROOT.TF1("gaussfit", "gaus", 81.e3, 101.e3) # Gaussian fit in the Z-peak range
dataHisto.Fit(gaussFit, "E") # Fit the function to the histogram using improved error treatment
dataHisto.Draw("pe")

# Add the fit results to the plot
latex = ROOT.TLatex() # Create the TLatex object
latex.SetNDC() # Set coordinates to be percent-based
latex.SetTextSize(0.03)
chi2 = gaussFit.GetChisquare()
ndof = gaussFit.GetNDF()
latex.DrawText(0.5, 0.80, "Mean = %.1f GeV" % (gaussFit.GetParameter(1) / 1000))
latex.DrawText(0.5, 0.75, "Width = %.1f GeV" % (gaussFit.GetParameter(2) / 1000))
latex.DrawText(0.5, 0.70, "chi2/ndof = %.1f/%d = %.1f" % (chi2, ndof, chi2 / ndof))
canvas.Print(plotFileName) # Write the plot to the output file

# Fit a relativistic Breit-Wigner to the Z peak
k_num = "2*sqrt(2)*[0]*[1]*sqrt([0]*[0]*([0]*[0]+[1]*[1]))"
k_denom = "3.14159*sqrt([0]*[0] + sqrt([0]*[0]*([0]*[0]+[1]*[1])))"
denom = "((x*x-[0]*[0])*(x*x-[0]*[0]) + [0]*[0]*[1]*[1])"
bwFit = ROOT.TF1("bwfit", "[2]*(%s/%s)/%s" % (k_num, k_denom, denom), 50.e3, 200.e3)
bwFit.SetParameter(0, 91.1876e3) # Mass of Z boson in MeV
bwFit.SetParameter(1, 2.4952e3) # Width of Z boson in MeV
dataHisto.Fit(bwFit, "E")
dataHisto.Draw("pe")
chi2 = bwFit.GetChisquare()
ndof = bwFit.GetNDF()
latex.DrawText(0.5, 0.80, "Mean = %.1f GeV" % (bwFit.GetParameter(0) / 1000))
latex.DrawText(0.5, 0.75, "Width = %.1f GeV" % (bwFit.GetParameter(1) / 1000))
latex.DrawText(0.5, 0.70, "chi2/ndof = %.1f/%d = %.1f" % (chi2, ndof, chi2 / ndof))
canvas.Print(plotFileName)

# Calculate the ratio
ratio = dataHisto.Clone()
ratio.Divide(bwFit)
ratio.SetLineColor(ROOT.kRed)

# Draw the normal plots (not the ratio)
pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 1)
pad1.SetLogy(True) # Set the y-axis of the top plot to be logarithmic
pad1.SetBottomMargin(0) # Upper and lower pads are joined
pad1.Draw() # Draw the upper pad in the canvas
pad1.cd() # pad1 becomes the current pad
dataHisto.SetTitle("") # Remove the plot title
dataHisto.GetXaxis().SetLabelSize(0) # Remove x-axis labels for the top pad
dataHisto.GetXaxis().SetTitleSize(0) # Remove x-axis title for the top pad
dataHisto.GetYaxis().SetTitleSize(0.05) # Increase y-axis title size (pad is not full page)
dataHisto.Draw("pe,same") # Draw the data points on top of the MC histo

# Add a legend to the top pad
legend = ROOT.TLegend(0.7, 0.6, 0.85, 0.75)
legend.AddEntry(dataHisto, "Data")
legend.AddEntry(bwFit, "Breit-Wigner Fit")
legend.SetLineWidth(0)
legend.Draw("same")

# Add other labels with TLatex
latex.SetTextSize(0.06)
latex.DrawText(0.7, 0.83, "ATLAS_2016_RUN")
latex.SetTextSize(0.04)
latex.DrawText(0.7, 0.77, "Di-muon events")

# Draw the mass, width, and chi2/ndof
latex.DrawText(0.4, 0.80, "Mean = %.1f GeV" % (bwFit.GetParameter(0) / 1000))
latex.DrawText(0.4, 0.75, "Width = %.1f GeV" % (bwFit.GetParameter(1) / 1000))
latex.DrawText(0.4, 0.70, "chi2/ndof = %.1f/%d = %.1f" % (chi2, ndof, chi2 / ndof))

# Now draw the ratio
canvas.cd() # Go back to the main canvas before defining pad2
pad2 = ROOT.TPad("pad2", "pad2", 0, 0.05, 1, 0.3)
pad2.SetTopMargin(0) # Upper and lower pads are joined
pad2.SetBottomMargin(0.25) # Expand the bottom margin for extra label space
pad2.Draw() # Draw the lower pad in the canvas
pad2.cd() # pad2 becomes the current pad
ratio.SetTitle("") # Turn off the title to avoid overlap
ratio.GetXaxis().SetLabelSize(0.12) # Larger x-axis labels (pad is not full page)
ratio.GetXaxis().SetTitleSize(0.12) # Larger x-axis title (pad is not full page)
ratio.GetYaxis().SetLabelSize(0.1) # Larger y-axis labels (pad is not full page)
ratio.GetYaxis().SetTitleSize(0.15) # Larger y-axis title (pad is not full page)
ratio.GetYaxis().SetTitle("Data/Fit") # Change the y-axis title (this is the ratio)
ratio.GetYaxis().SetTitleOffset(0.3) # Reduce the y-axis title spacing
ratio.GetYaxis().SetRangeUser(0.5, 1.5) # Set the y-axis ratio range from 0.5 to 1.5
ratio.GetYaxis().SetNdivisions(207) # Change the y-axis tick-marks to work better
ratio.SetMarkerStyle(3) # Change the marker style to stars
ratio.SetMarkerColor(ROOT.kRed) # Change the marker colour to red
ratio.Draw("pe") # Draw the ratio in the current pad, without errors

# Add a line at 1 to the ratio plot
line = ROOT.TLine(50.e3, 1, 200.e3, 1) # Draw a line at 1 from 50 GeV to 200 GeV (full plot)
line.SetLineColor(ROOT.kBlack) # Set the line colour to black
line.SetLineWidth(2) # Set the line width to 2
line.Draw("same") # Draw the line on the same plot as the ratio

# Write the ratio plot to the output plot file
canvas.Print(plotFileName)

# Close the output file
canvas.Print(plotFileName + "]")


 FCN=10889 FROM MINOS     STATUS=SUCCESSFUL     20 CALLS         225 TOTAL
                     EDM=6.81397e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     6.26913e+03   4.30927e+01   5.26908e-02  -2.92835e-06
   2  Mean         9.06248e+04   1.47209e+01  -1.18182e-02  -7.20941e-07
   3  Sigma        3.29138e+03   1.73890e+01   1.73890e+01  -4.18851e-02
 FCN=1841.93 FROM MINOS     STATUS=SUCCESSFUL     20 CALLS         264 TOTAL
                     EDM=3.56586e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           9.06734e+04   1.47171e+01   3.92787e-03  -5.33917e-07
   2  p1           4.82007e+03   2.64479e+01  -3.86546e-03   1.03413e-07
   3  p2           3.78873e-03   1.

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ATLAS_2016_RUN_fitting.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file ATLAS_2016_RUN_fitting.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file ATLAS_2016_RUN_fitting.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file ATLAS_2016_RUN_fitting.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file ATLAS_2016_RUN_fitting.pdf
Info in <TCanvas::Print>: pdf file ATLAS_2016_RUN_fitting.pdf has been closed
